# Setup Azure Cognitive Search Service
This notebook will set up your Azure Cognitive Search Service.  

First, you will need an Azure account.  If you don't already have one, you can start a free trial of Azure [here](https://azure.microsoft.com/free/).  

Secondly, create a new Azure search service using the Azure portal at <https://portal.azure.com/#create/Microsoft.Search>.  Select your Azure subscription.  You may create a new resource group (you can name it something like "covid19-search-rg").  You will need a globally-unique URL as the name of your search service (try something like "covid19-search-" plus your name, organization, or numbers).  Finally, choose a nearby location to host your search service - please remember the location that you chose, as your Cognitive Services instance will need to be based in the same location.  Click "Review + create" and then (after validation) click "Create" to instantiate and deploy the service.  

After deployment is complete, click "Go to resource" to navigate to your new search service.  We will need some information about your search service to fill in the "Azure Search variables" section in the cell below.  First, on the "Overview" main page, you should see a "Url" value.  Copy that value into the "azsearch_url" variable in the cell below (you can just update the "<YourSearchServiceName>" section of the URL with the name of your Azure search service).  Then, on the Azure portal page in the left-hand pane under "Settings", click on "Keys".  Update the azsearch_key value below with one of the keys from your service on the Azure portal page.    

In [ ]:
# Azure Search variables
azsearch_url = "<YourSearchServiceName>.search.windows.net"  # If you copy this value from the portal, leave off the "https://" from the beginning
azsearch_key = "TODO" 

# Cognitive Services variables


# Data source which contains documents to process
blob_connection_string = ""
blob_container = "small"

# Prefix for elements of the Cognitive Search service
search_prefix = "pwc"

print("The variables are initialized.")

TODO: explain

In [ ]:
import json

# Function to wrap REST requests to Azure Search service
def azsearch_rest(request_type="GET", endpoint="servicestats", body=None):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }

    # Request parameters
    params = urllib.parse.urlencode({
        'api-version':'2019-05-06-Preview'
    })
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/{0}?{1}".format(endpoint, params)
        conn.request(request_type, request_path, body, headers)
        response = conn.getresponse()
        print(response.status)
        data = response.read().decode("UTF-8")
        result = None
        if len(data) > 0:
            result = json.loads(data)
        return result

    except Exception as ex:
        raise ex
        
# Test the function
try:
    response = azsearch_rest()
    if response != None:
            print(json.dumps(response, sort_keys=True, indent=2))
except Exception as ex:
    print(ex.message)

First, let's set up the data source for your service.  

In [ ]:
datasource_name = search_prefix + "-ds"

# Define the request body with details of the data source we want to create
body = {   
    "name": datasource_name,  
    "description": "",  
    "type": "azureblob",
    "credentials":
        { "connectionString": blob_connection_string
        },  
    "container": { "name": blob_container }
} 

try:
    # Call the REST API's 'datasources' endpoint to create a data sourve
    result = azsearch_rest(request_type="POST", endpoint="datasources", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
except Exception as ex:
    print(ex)

Then let's set up your search index.  

In [ ]:
index_name = search_prefix + "-index"

# Define the request body
with open("index.json") as datafile:
  index_json = json.load(datafile)

try:
    result = azsearch_rest(request_type="PUT", endpoint="indexes/" + index_name, body=json.dumps(index_json))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)